# SAC Season by Season Analyzer

notebook to analyze SAC seasons, and specifically the cali crown rivalry.

In [43]:
import os, sys, subprocess, json, time
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
xkcd_colors = sns.xkcd_rgb
GOLLY_GRAY = "#272B30"

In [50]:
CUP = 'hellmouth'
ABBR = 'SAC'

## Loading Match Data

Start by loading match data from the `data/` dir:

In [46]:
def run_cmd(cmd_str):
    child = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE)
    output = child.communicate()[0].decode()
    return output

print(run_cmd('ls ../data/'))

clone.sh
gollyx-dragon-data
gollyx-hellmouth-data
gollyx-klein-data
gollyx-pseudo-data
gollyx-rainbow-data
gollyx-star-data
gollyx-toroidal-data



## Method to load regular season data

user specifies the cup and the season.

In [47]:
def fetch_season_data(which_season0):
    seas_file = os.path.join('..', 'data', f'gollyx-{CUP}-data', f'season{which_season0}', 'season.json')
    if not os.path.exists(seas_file):
        raise Exception(f"Error: season {which_season0} not valid: {seas_file} does not exist")
    with open(seas_file, 'r') as f:
        season0_seas = json.load(f)
    return season0_seas

## Method to filter regular season data on a single team

In [48]:
def filter_season_data(seas_data, team_abbr):
    team_abbr = team_abbr.upper()
    team_seas = []
    for day in seas_data:
        for game in day:
            if game['team1Abbr'] == team_abbr or game['team2Abbr'] == team_abbr:
                team_seas.append(game)
                break
    return team_seas

## Test drive

In [51]:
sac = filter_season_data(fetch_season_data(3), ABBR)
print(len(sac))
pprint(sac[0])

49
{'day': 0,
 'gameid': '5b2ff315-22e5-4546-af93-3a83557e97a3',
 'generations': 1821,
 'isPostseason': False,
 'league': 'Hot League',
 'map': {'cellSize': 7,
         'columns': 120,
         'initialConditions1': '[{"25":[89,90,93,94,95]},{"26":[92]},{"27":[90]}]',
         'initialConditions2': '[{"75":[27]},{"76":[29]},{"77":[26,27,30,31,32]}]',
         'mapName': 'The Quad',
         'mapZone1Name': 'Quadrant I',
         'mapZone2Name': 'Quadrant II',
         'mapZone3Name': 'Quadrant III',
         'mapZone4Name': 'Quadrant IV',
         'patternName': 'twoacorn',
         'rows': 100,
         'url': '?s1=[{"25":[89,90,93,94,95]},{"26":[92]},{"27":[90]}]&s2=[{"75":[27]},{"76":[29]},{"77":[26,27,30,31,32]}]'},
 'season': 3,
 'team1Abbr': 'DET',
 'team1Color': '#9963ab',
 'team1Name': 'Detroit Grape Chews',
 'team1Score': 286,
 'team1WinLoss': [0, 0],
 'team2Abbr': 'SAC',
 'team2Color': '#ffb627',
 'team2Name': 'Sacramento Boot Lickers',
 'team2Score': 54,
 'team2WinLoss': [0,

## Method to assemble schedule tuples - opponent, outcome, score

Extract data from the full game data structure to compile a simpler tuple with opponent, outcome, and score.

In [52]:
def schedule_tuples(team_abbr, season0):
    team_abbr = team_abbr.upper()
    dat = filter_season_data(fetch_season_data(season0), team_abbr)
    schedule_tups = []
    for game in dat:
        if game['team1Abbr'] == team_abbr:
            other_team_name = game['team2Name']
            other_team_abbr = game['team2Abbr']
            
            our_points = game['team1Score']
            other_points = game['team2Score']
            
            our_wl = game['team1WinLoss']
            other_wl = game['team2WinLoss']
            
            if game['team2Score'] > game['team1Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'H'

        elif game['team2Abbr'] == team_abbr:
            other_team_name = game['team1Name']
            other_team_abbr = game['team1Abbr']
            
            our_points = game['team2Score']
            other_points = game['team1Score']
            
            our_wl = game['team2WinLoss']
            other_wl = game['team1WinLoss']
            
            if game['team1Score'] > game['team2Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'A'
            
        else:
            raise Exception(f"Could not find team {team_abbr} in season {season0}")

        tup = (other_team_name, other_team_abbr, homeaway, outcome, our_points, other_points, our_wl, other_wl)
        schedule_tups.append(tup)
    return schedule_tups

In [53]:
def print_sched_tuples(team_abbr, season0):
    tups = schedule_tuples(team_abbr, season0)
    print("")
    print(f"Schedule for {team_abbr.upper()}, Season {season0+1}")
    for tup in tups:
        (oppname, oppabbr, homeaway, outcome, ourpts, theirpts, ourwl, theirwl) = tup
        if homeaway=='H':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2}) vs {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        elif homeaway=='A':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2})  @ {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        print(f"    {matchup}\t{outcome:4}\t{ourpts:4} - {theirpts:4}")
    print("")

In [54]:
print_sched_tuples(ABBR, 3)


Schedule for SAC, Season 4
    SAC  ( 0 -  0)  @ DET  ( 0 -  0)	L   	  54 -  286
    SAC  ( 0 -  1) vs DET  ( 1 -  0)	W   	 157 -   57
    SAC  ( 1 -  1)  @ DECO ( 1 -  1)	W   	 143 -  123
    SAC  ( 2 -  1) vs DECO ( 1 -  2)	L   	  75 -  100
    SAC  ( 2 -  2)  @ DECO ( 2 -  2)	L   	 137 -  203
    SAC  ( 2 -  3) vs DECO ( 3 -  2)	W   	 180 -  103
    SAC  ( 3 -  3)  @ MILF ( 2 -  4)	L   	  72 -  232
    SAC  ( 3 -  4) vs MILF ( 3 -  4)	L   	  90 -  199
    SAC  ( 3 -  5)  @ SFBS ( 3 -  5)	W   	 105 -   74
    SAC  ( 4 -  5) vs SFBS ( 3 -  6)	W   	 299 -   14
    SAC  ( 5 -  5)  @ SFBS ( 3 -  7)	W   	 205 -  104
    SAC  ( 6 -  5) vs SFBS ( 3 -  8)	L   	  74 -  197
    SAC  ( 6 -  6)  @ MILF ( 5 -  7)	L   	  63 -   67
    SAC  ( 6 -  7) vs MILF ( 6 -  7)	W   	 127 -   67
    SAC  ( 7 -  7)  @ MILF ( 6 -  8)	L   	  82 -  127
    SAC  ( 7 -  8) vs MILF ( 7 -  8)	L   	  82 -  227
    SAC  ( 7 -  9)  @ MILF ( 8 -  8)	W   	 190 -   44
    SAC  ( 8 -  9) vs MILF ( 8 -  9)	L   	  39 -  254


In [72]:
print_sched_tuples(ABBR, 13)


Schedule for SAC, Season 14
    SAC  ( 0 -  0)  @ MILF ( 0 -  0)	W   	 124 -   72
    SAC  ( 1 -  0) vs MILF ( 0 -  1)	W   	 340 -   82
    SAC  ( 2 -  0)  @ DET  ( 1 -  1)	L   	  42 -  236
    SAC  ( 2 -  1) vs DET  ( 2 -  1)	L   	 146 -  189
    SAC  ( 2 -  2)  @ DET  ( 3 -  1)	W   	 142 -  124
    SAC  ( 3 -  2) vs DET  ( 3 -  2)	L   	 112 -  201
    SAC  ( 3 -  3)  @ MILF ( 1 -  5)	W   	 230 -    3
    SAC  ( 4 -  3) vs MILF ( 1 -  6)	W   	 168 -  140
    SAC  ( 5 -  3)  @ MILF ( 1 -  7)	L   	 130 -  194
    SAC  ( 5 -  4) vs MILF ( 2 -  7)	L   	 154 -  210
    SAC  ( 5 -  5)  @ SFBS ( 5 -  5)	L   	   0 -  509
    SAC  ( 5 -  6) vs SFBS ( 6 -  5)	L   	 142 -  193
    SAC  ( 5 -  7)  @ DET  ( 8 -  4)	L   	  92 -  282
    SAC  ( 5 -  8) vs DET  ( 9 -  4)	W   	 147 -  103
    SAC  ( 6 -  8)  @ DECO ( 6 -  8)	L   	  43 -  158
    SAC  ( 6 -  9) vs DECO ( 7 -  8)	W   	 295 -  121
    SAC  ( 7 -  9)  @ SFBS ( 8 -  8)	L   	  13 -  146
    SAC  ( 7 - 10) vs SFBS ( 9 -  8)	L   	  52 -  318

In [73]:
print_sched_tuples(ABBR, 20)


Schedule for SAC, Season 21
    SAC  ( 0 -  0)  @ ABQ  ( 0 -  0)	W   	 154 -  123
    SAC  ( 1 -  0) vs ABQ  ( 0 -  1)	W   	 291 -   20
    SAC  ( 2 -  0) vs BB   ( 0 -  2)	W   	 218 -  187
    SAC  ( 3 -  0)  @ BB   ( 0 -  3)	W   	 145 -   70
    SAC  ( 4 -  0)  @ ABQ  ( 1 -  3)	L   	 111 -  184
    SAC  ( 4 -  1) vs ABQ  ( 2 -  3)	L   	  43 -  255
    SAC  ( 4 -  2)  @ SFBS ( 3 -  3)	W   	 244 -   41
    SAC  ( 5 -  2) vs SFBS ( 3 -  4)	W   	  74 -   50
    SAC  ( 6 -  2)  @ DET  ( 5 -  3)	W   	 209 -  129
    SAC  ( 7 -  2) vs DET  ( 5 -  4)	L   	 105 -  235
    SAC  ( 7 -  3)  @ ORL  ( 8 -  2)	W   	 155 -   55
    SAC  ( 8 -  3) vs ORL  ( 8 -  3)	W   	 122 -  102
    SAC  ( 9 -  3)  @ ORL  ( 8 -  4)	W   	 193 -   80
    SAC  (10 -  3) vs ORL  ( 8 -  5)	W   	 175 -   98
    SAC  (11 -  3)  @ MILF ( 9 -  5)	W   	 185 -  131
    SAC  (12 -  3) vs MILF ( 9 -  6)	W   	 168 -  148
    SAC  (13 -  3) vs ABQ  ( 7 -  9)	W   	 178 -  163
    SAC  (14 -  3)  @ ABQ  ( 7 - 10)	L   	 112 -  167

## Method to assemble matchup tuples - specific opponent, outcome, score

In [74]:
def matchup_tuples(team_abbr, opp_abbr, season0):
    team_abbr, opp_abbr = team_abbr.upper(), opp_abbr.upper()
    dat = filter_season_data(fetch_season_data(season0), team_abbr)
    matchup_tups = []
    for game in dat:
        skip = False
        if game['team1Abbr'] == team_abbr and game['team2Abbr'] == opp_abbr:
            other_team_name = game['team2Name']
            other_team_abbr = game['team2Abbr']
            
            our_points = game['team1Score']
            other_points = game['team2Score']
            
            our_wl = game['team1WinLoss']
            other_wl = game['team2WinLoss']
            
            if game['team2Score'] > game['team1Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'H'

        elif game['team2Abbr'] == team_abbr and game['team1Abbr'] == opp_abbr:
            other_team_name = game['team1Name']
            other_team_abbr = game['team1Abbr']
            
            our_points = game['team2Score']
            other_points = game['team1Score']
            
            our_wl = game['team2WinLoss']
            other_wl = game['team1WinLoss']
            
            if game['team1Score'] > game['team2Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'A'
            
        else:
            skip = True
            
        if not skip:
            tup = (other_team_name, other_team_abbr, homeaway, outcome, our_points, other_points, our_wl, other_wl)
            matchup_tups.append(tup)
    return matchup_tups

In [75]:
def print_matchup_tuples(team_abbr, opp_abbr, season0):
    tups = matchup_tuples(team_abbr, opp_abbr, season0)
    print("")
    print(f"Schedule for {team_abbr.upper()} vs. {opp_abbr.upper()}, Season {season0+1}")
    wins = sum([1 for tup in tups if tup[3]=='W'])
    losses = sum([1 for tup in tups if tup[3]=='L'])
    print(f"W-L record for {team_abbr}: {wins}-{losses}")
    for tup in tups:
        (oppname, oppabbr, homeaway, outcome, ourpts, theirpts, ourwl, theirwl) = tup
        if homeaway=='H':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2}) vs {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        elif homeaway=='A':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2})  @ {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        print(f"    {matchup}\t{outcome:4}\t{ourpts:4} - {theirpts:4}")
    print("")

## Cali Crown SAC-SFBS

In [76]:
print_matchup_tuples(ABBR, 'SFBS', 3)


Schedule for SAC vs. SFBS, Season 4
W-L record for SAC: 5-3
    SAC  ( 3 -  5)  @ SFBS ( 3 -  5)	W   	 105 -   74
    SAC  ( 4 -  5) vs SFBS ( 3 -  6)	W   	 299 -   14
    SAC  ( 5 -  5)  @ SFBS ( 3 -  7)	W   	 205 -  104
    SAC  ( 6 -  5) vs SFBS ( 3 -  8)	L   	  74 -  197
    SAC  ( 8 - 10)  @ SFBS ( 9 -  9)	W   	 125 -   45
    SAC  ( 9 - 10) vs SFBS ( 9 - 10)	W   	 264 -   91
    SAC  (10 - 10)  @ SFBS ( 9 - 11)	L   	  35 -  211
    SAC  (10 - 11) vs SFBS (10 - 11)	L   	 136 -  150



In [77]:
for season0 in range(0, 3):
    print_matchup_tuples(ABBR, 'SFBS', season0)


Schedule for SAC vs. SFBS, Season 1
W-L record for SAC: 2-4
    SAC  ( 8 -  8) vs SFBS (11 -  5)	L   	  39 -   78
    SAC  ( 8 -  9)  @ SFBS (12 -  5)	L   	  29 -  220
    SAC  ( 8 - 10)  @ SFBS (13 -  5)	L   	  11 -  173
    SAC  ( 8 - 11) vs SFBS (14 -  5)	W   	 161 -   88
    SAC  (12 - 12)  @ SFBS (16 -  8)	L   	  55 -  205
    SAC  (12 - 13) vs SFBS (17 -  8)	W   	 136 -  109


Schedule for SAC vs. SFBS, Season 2
W-L record for SAC: 2-4
    SAC  ( 0 -  0)  @ SFBS ( 0 -  0)	L   	   6 -  319
    SAC  ( 0 -  1) vs SFBS ( 1 -  0)	W   	 162 -  104
    SAC  ( 3 -  5)  @ SFBS ( 3 -  5)	L   	  94 -  239
    SAC  ( 3 -  6) vs SFBS ( 4 -  5)	L   	  95 -  136
    SAC  (10 - 14) vs SFBS (10 - 14)	W   	 128 -   59
    SAC  (11 - 14)  @ SFBS (10 - 15)	L   	  46 -   85


Schedule for SAC vs. SFBS, Season 3
W-L record for SAC: 3-3
    SAC  ( 6 -  6)  @ SFBS ( 5 -  7)	L   	  51 -  244
    SAC  ( 6 -  7) vs SFBS ( 6 -  7)	W   	 270 -   58
    SAC  (11 -  7) vs SFBS ( 8 - 10)	W   	 162 -   45
    S

In [78]:
for season0 in range(3, 24):
    print_matchup_tuples(ABBR, 'SFBS', season0)


Schedule for SAC vs. SFBS, Season 4
W-L record for SAC: 5-3
    SAC  ( 3 -  5)  @ SFBS ( 3 -  5)	W   	 105 -   74
    SAC  ( 4 -  5) vs SFBS ( 3 -  6)	W   	 299 -   14
    SAC  ( 5 -  5)  @ SFBS ( 3 -  7)	W   	 205 -  104
    SAC  ( 6 -  5) vs SFBS ( 3 -  8)	L   	  74 -  197
    SAC  ( 8 - 10)  @ SFBS ( 9 -  9)	W   	 125 -   45
    SAC  ( 9 - 10) vs SFBS ( 9 - 10)	W   	 264 -   91
    SAC  (10 - 10)  @ SFBS ( 9 - 11)	L   	  35 -  211
    SAC  (10 - 11) vs SFBS (10 - 11)	L   	 136 -  150


Schedule for SAC vs. SFBS, Season 5
W-L record for SAC: 3-1
    SAC  ( 3 -  1)  @ SFBS ( 1 -  3)	W   	 214 -  199
    SAC  ( 4 -  1) vs SFBS ( 1 -  4)	L   	 111 -  160
    SAC  ( 4 -  4)  @ SFBS ( 3 -  5)	W   	 151 -   86
    SAC  ( 5 -  4) vs SFBS ( 3 -  6)	W   	 236 -   72


Schedule for SAC vs. SFBS, Season 6
W-L record for SAC: 4-4
    SAC  ( 4 -  4)  @ SFBS ( 3 -  5)	L   	  98 -  266
    SAC  ( 4 -  5) vs SFBS ( 4 -  5)	L   	 142 -  217
    SAC  ( 4 - 10)  @ SFBS ( 8 -  6)	L   	 100 -  153
    S

## Boston-Sacramento

east coast west coast, not much fuel for a rivalry

### Alewife-Sacramento AA-SAC

In [81]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'AA', season0)


Schedule for SAC vs. AA, Season 1
W-L record for SAC: 3-1
    SAC  (17 - 15)  @ AA   (13 - 19)	W   	 144 -  128
    SAC  (18 - 15) vs AA   (13 - 20)	W   	 118 -  104
    SAC  (23 - 23) vs AA   (21 - 25)	W   	 159 -  143
    SAC  (24 - 23)  @ AA   (21 - 26)	L   	 102 -  130


Schedule for SAC vs. AA, Season 2
W-L record for SAC: 3-5
    SAC  (16 - 18)  @ AA   (19 - 15)	L   	 144 -  204
    SAC  (16 - 19) vs AA   (20 - 15)	W   	 145 -  107
    SAC  (17 - 19) vs AA   (20 - 16)	W   	 438 -   34
    SAC  (18 - 19)  @ AA   (20 - 17)	L   	  28 -  381
    SAC  (21 - 23) vs AA   (25 - 19)	L   	  54 -  170
    SAC  (21 - 24)  @ AA   (26 - 19)	L   	  95 -  163
    SAC  (21 - 25) vs AA   (27 - 19)	L   	  70 -  177
    SAC  (21 - 26)  @ AA   (28 - 19)	W   	 112 -   81


Schedule for SAC vs. AA, Season 3
W-L record for SAC: 3-3
    SAC  (15 - 13) vs AA   (12 - 16)	W   	 111 -   64
    SAC  (16 - 13)  @ AA   (12 - 17)	L   	   0 -  472
    SAC  (18 - 20) vs AA   (16 - 22)	W   	 230 -   37
    SAC  (1

### Boylston-Sacramento BB-SAC

In [80]:
for season0 in range(13, 24):
    print_matchup_tuples(ABBR, 'BB', season0)


Schedule for SAC vs. BB, Season 14
W-L record for SAC: 6-2
    SAC  (17 - 17) vs BB   (19 - 15)	W   	 172 -  149
    SAC  (18 - 17)  @ BB   (19 - 16)	W   	 146 -   23
    SAC  (21 - 17)  @ BB   (21 - 17)	W   	 239 -  111
    SAC  (22 - 17) vs BB   (21 - 18)	W   	 216 -  166
    SAC  (23 - 17) vs BB   (21 - 19)	W   	 181 -  152
    SAC  (24 - 17)  @ BB   (21 - 20)	L   	  77 -  270
    SAC  (25 - 19) vs BB   (22 - 22)	W   	 164 -  161
    SAC  (26 - 19)  @ BB   (22 - 23)	L   	  84 -  257


Schedule for SAC vs. BB, Season 15
W-L record for SAC: 5-5
    SAC  ( 0 -  0) vs BB   ( 0 -  0)	L   	  99 -  191
    SAC  ( 0 -  1)  @ BB   ( 1 -  0)	W   	 133 -   90
    SAC  (13 - 11)  @ BB   (11 - 13)	L   	 102 -  188
    SAC  (13 - 12) vs BB   (12 - 13)	W   	 189 -  183
    SAC  (14 - 14)  @ BB   (14 - 14)	W   	 245 -  102
    SAC  (15 - 14) vs BB   (14 - 15)	L   	 117 -  169
    SAC  (15 - 15) vs BB   (15 - 15)	W   	 239 -   46
    SAC  (16 - 15)  @ BB   (15 - 16)	W   	 117 -   49
    SAC  (18 - 

## Detroit-Sacramento

grape chews - boot lickers

chews vs lickers

Wario Crown

short stolid scores, double digit vs triple digit

not sure if this is true of all rivalries... but some seasons is just hemming in rival to two digit scores each time

In [82]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'DET', season0)


Schedule for SAC vs. DET, Season 1
W-L record for SAC: 4-3
    SAC  ( 4 -  2) vs DET  ( 4 -  2)	W   	  90 -   21
    SAC  ( 5 -  2)  @ DET  ( 4 -  3)	L   	  25 -  100
    SAC  ( 5 -  3)  @ DET  ( 5 -  3)	L   	  13 -  296
    SAC  ( 5 -  4) vs DET  ( 6 -  3)	W   	 308 -   37
    SAC  ( 7 -  7) vs DET  ( 9 -  5)	W   	 151 -   36
    SAC  ( 8 -  7)  @ DET  ( 9 -  6)	L   	  64 -  152
    SAC  (24 - 24)  @ DET  (26 - 22)	W   	 244 -   55


Schedule for SAC vs. DET, Season 2
W-L record for SAC: 2-4
    SAC  ( 1 -  1) vs DET  ( 0 -  2)	W   	 302 -    0
    SAC  ( 2 -  1)  @ DET  ( 0 -  3)	L   	   0 -  244
    SAC  ( 4 -  8) vs DET  ( 5 -  7)	W   	 292 -   76
    SAC  ( 5 -  8)  @ DET  ( 5 -  8)	L   	  66 -  166
    SAC  (11 - 15) vs DET  (11 - 15)	L   	  52 -  218
    SAC  (11 - 16)  @ DET  (12 - 15)	L   	  95 -   97


Schedule for SAC vs. DET, Season 3
W-L record for SAC: 2-4
    SAC  ( 1 -  1) vs DET  ( 0 -  2)	W   	 241 -   54
    SAC  ( 2 -  1)  @ DET  ( 0 -  3)	L   	   0 -  127
    SAC 


Schedule for SAC vs. DET, Season 23
W-L record for SAC: 2-2
    SAC  ( 2 -  2)  @ DET  ( 2 -  2)	W   	 137 -   48
    SAC  ( 3 -  2) vs DET  ( 2 -  3)	L   	  21 -  170
    SAC  (15 - 23)  @ DET  (17 - 21)	W   	 220 -  180
    SAC  (16 - 23) vs DET  (17 - 22)	L   	  55 -  173


Schedule for SAC vs. DET, Season 24
W-L record for SAC: 3-5
    SAC  ( 5 -  5)  @ DET  ( 6 -  4)	L   	  57 -  260
    SAC  ( 5 -  6) vs DET  ( 7 -  4)	W   	 287 -   49
    SAC  (15 - 15)  @ DET  (18 - 12)	L   	 104 -  156
    SAC  (15 - 16) vs DET  (19 - 12)	W   	 320 -   48
    SAC  (17 - 17)  @ DET  (20 - 14)	W   	 231 -  137
    SAC  (18 - 17) vs DET  (20 - 15)	L   	 114 -  165
    SAC  (20 - 20)  @ DET  (23 - 17)	L   	  85 -  283
    SAC  (20 - 21) vs DET  (24 - 17)	L   	 158 -  200



## Milwaukee-Sacramento MILF-SAC



In [83]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'MILF', season0)


Schedule for SAC vs. MILF, Season 1
W-L record for SAC: 5-3
    SAC  ( 1 -  1) vs MILF ( 1 -  1)	W   	 230 -   36
    SAC  ( 2 -  1)  @ MILF ( 1 -  2)	W   	 102 -   27
    SAC  ( 3 -  1)  @ MILF ( 1 -  3)	L   	  54 -   99
    SAC  ( 3 -  2) vs MILF ( 2 -  3)	W   	 158 -  156
    SAC  ( 7 -  5) vs MILF ( 6 -  6)	L   	  86 -   93
    SAC  ( 7 -  6)  @ MILF ( 7 -  6)	L   	  55 -   82
    SAC  (10 - 12)  @ MILF (12 - 10)	W   	 129 -   88
    SAC  (11 - 12) vs MILF (12 - 11)	W   	 117 -   63


Schedule for SAC vs. MILF, Season 2
W-L record for SAC: 4-4
    SAC  ( 3 -  7)  @ MILF ( 4 -  6)	L   	  25 -  219
    SAC  ( 3 -  8) vs MILF ( 5 -  6)	W   	 260 -   55
    SAC  ( 7 - 11) vs MILF ( 9 -  9)	W   	 173 -   20
    SAC  ( 8 - 11)  @ MILF ( 9 - 10)	L   	  27 -  339
    SAC  ( 8 - 12)  @ MILF (10 - 10)	L   	 140 -  147
    SAC  ( 8 - 13) vs MILF (11 - 10)	W   	 186 -   75
    SAC  ( 9 - 13) vs MILF (11 - 11)	W   	 219 -   55
    SAC  (10 - 13)  @ MILF (11 - 12)	L   	  29 -  299


Schedule fo


Schedule for SAC vs. MILF, Season 22
W-L record for SAC: 5-5
    SAC  ( 7 -  5)  @ MILF ( 8 -  4)	W   	 124 -   43
    SAC  ( 8 -  5) vs MILF ( 8 -  5)	L   	  47 -   78
    SAC  ( 9 - 13)  @ MILF (13 -  9)	L   	 134 -  153
    SAC  ( 9 - 14) vs MILF (14 -  9)	W   	 238 -   67
    SAC  (12 - 16)  @ MILF (15 - 13)	L   	 131 -  141
    SAC  (12 - 17) vs MILF (16 - 13)	W   	 182 -  131
    SAC  (17 - 23)  @ MILF (19 - 21)	W   	 101 -   88
    SAC  (18 - 23) vs MILF (19 - 22)	W   	 216 -  108
    SAC  (19 - 27)  @ MILF (20 - 26)	L   	 107 -  138
    SAC  (19 - 28) vs MILF (21 - 26)	L   	  69 -  252


Schedule for SAC vs. MILF, Season 23
W-L record for SAC: 3-7
    SAC  ( 0 -  0)  @ MILF ( 0 -  0)	L   	  93 -  228
    SAC  ( 0 -  1) vs MILF ( 1 -  0)	W   	 179 -   59
    SAC  ( 7 -  7)  @ MILF ( 6 -  8)	L   	  73 -  199
    SAC  ( 7 -  8) vs MILF ( 7 -  8)	L   	   8 -  181
    SAC  (10 - 10)  @ MILF ( 9 - 11)	L   	 127 -  185
    SAC  (10 - 11) vs MILF (10 - 11)	L   	  93 -  172
    SAC  (1

## Orlando-Sacramento ORL-SAC

I mean, what's even... not even... what?


In [84]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'ORL', season0)


Schedule for SAC vs. ORL, Season 1
W-L record for SAC: 4-6
    SAC  (19 - 15)  @ ORL  (16 - 18)	L   	  50 -  147
    SAC  (19 - 16) vs ORL  (17 - 18)	L   	  74 -  102
    SAC  (19 - 17) vs ORL  (18 - 18)	W   	 314 -   62
    SAC  (20 - 17)  @ ORL  (18 - 19)	L   	  39 -  243
    SAC  (20 - 20)  @ ORL  (19 - 21)	L   	  99 -  125
    SAC  (20 - 21) vs ORL  (20 - 21)	W   	 244 -   24
    SAC  (21 - 21) vs ORL  (20 - 22)	W   	 181 -   24
    SAC  (22 - 21)  @ ORL  (20 - 23)	L   	  23 -  126
    SAC  (22 - 22) vs ORL  (21 - 23)	L   	  51 -  169
    SAC  (22 - 23)  @ ORL  (22 - 23)	W   	 150 -  126


Schedule for SAC vs. ORL, Season 2
W-L record for SAC: 4-4
    SAC  (11 - 17)  @ ORL  (17 - 11)	L   	 143 -  244
    SAC  (11 - 18) vs ORL  (18 - 11)	W   	 431 -   13
    SAC  (14 - 18)  @ ORL  (19 - 13)	W   	 147 -   61
    SAC  (15 - 18) vs ORL  (19 - 14)	W   	 255 -    9
    SAC  (20 - 20)  @ ORL  (23 - 17)	L   	  60 -  135
    SAC  (20 - 21) vs ORL  (24 - 17)	W   	 161 -  101
    SAC  (21 - 


Schedule for SAC vs. ORL, Season 23
W-L record for SAC: 4-2
    SAC  ( 7 -  5) vs ORL  ( 8 -  4)	L   	  78 -  223
    SAC  ( 7 -  6)  @ ORL  ( 9 -  4)	L   	 146 -  210
    SAC  ( 7 -  9) vs ORL  (12 -  4)	W   	 190 -  171
    SAC  ( 8 -  9)  @ ORL  (12 -  5)	W   	 133 -   69
    SAC  (18 - 26) vs ORL  (24 - 20)	W   	 183 -   90
    SAC  (19 - 26)  @ ORL  (24 - 21)	W   	 141 -   46


Schedule for SAC vs. ORL, Season 24
W-L record for SAC: 1-7
    SAC  ( 1 -  1)  @ ORL  ( 2 -  0)	L   	  82 -  106
    SAC  ( 1 -  2) vs ORL  ( 3 -  0)	L   	  50 -  161
    SAC  ( 3 -  3) vs ORL  ( 5 -  1)	L   	 145 -  163
    SAC  ( 3 -  4)  @ ORL  ( 6 -  1)	W   	 206 -  176
    SAC  ( 6 -  6) vs ORL  ( 8 -  4)	L   	  56 -  165
    SAC  ( 6 -  7)  @ ORL  ( 9 -  4)	L   	  97 -  105
    SAC  (18 - 18)  @ ORL  (17 - 19)	L   	  57 -   90
    SAC  (18 - 19) vs ORL  (18 - 19)	L   	 131 -  209

